In [ ]:
import os
from platform import python_version
print(python_version())

In [ ]:
def rgb2bgr(input_img):
    bgr_image = np.zeros_like(input_img)
    bgr_image[:,:,0] = input_img[:,:,2] # R channel to B channel
    bgr_image[:,:,1] = input_img[:,:,1] # G channel remains the same
    bgr_image[:,:,2] = input_img[:,:,0] # B channel to R channel
    return bgr_image

In [ ]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
plt.style.use("ggplot")
%matplotlib inline
import gc

from tqdm.notebook import trange, tqdm

from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

import tensorflow as tf

print(tf.__version__)
print(tf.config.list_physical_devices())

from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.layers import Conv2D, Input, MaxPooling2D, Dropout, concatenate, UpSampling2D, ReLU
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, TensorBoard
from tensorflow.keras import backend as K

  
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.models import *
from tensorflow.keras.preprocessing.image import *
from tensorflow.keras.callbacks import *

import random 

## Seeding 
seed = 2019
random.seed = seed
np.random.seed = seed
tf.seed = seed

In [ ]:
w, h = 320,240
base_data_dir = "A:/data/hackathon_2023_1"
model_dir = "A:/models/tool_segmentation"
sample_test_dir = "A:/python_ai_projects/tool_segmentation/sample_data"
image_directory_name = "foreground"
mask_directory_name = "mask2"
background_dir_name = "background"
model_name = "mobilenet"

# **Directory reference, data augmentor, data loading**

In [ ]:
no_bg_change_list_file = os.path.join(base_data_dir, 'no_background_change.txt')
file_names_to_exclude_from_bg_change = []
if os.path.exists(no_bg_change_list_file):
    with open(no_bg_change_list_file, 'r') as f:
        # Read the contents of the file into a list of strings
        lines = f.readlines()

    # Strip the newline character from each line
    lines = [line.strip() for line in lines]


    file_names_to_exclude_from_bg_change = lines
print(file_names_to_exclude_from_bg_change)

def revert_preprocess(in_im):
    input_im = in_im.copy()
    input_im += 1
    input_im *= 127.5
    return input_im.astype(np.uint8)    

In [ ]:
import imgaug as ia
from imgaug import augmenters as iaa
from imgaug.augmentables.segmaps import SegmentationMapsOnImage

def activator_labels(images, augmenter, parents, default):
    if augmenter.name in ["AddToHueAndSaturation", "GaussianBlur"]: # "noaugment"
        return False
    else:
        return default
sometimes = lambda aug: iaa.Sometimes(0.25, aug)

seq = iaa.Sequential(
            [
            # apply the following augmenters to most images
            iaa.GaussianBlur(sigma=(0,0.05), name= "GaussianBlur"),
            iaa.Fliplr(0.5), # horizontally flip 50% of all images
            iaa.Flipud(0.5), # vertically flip 20% of all images
            # crop images by -5% to 10% of their height/width
            sometimes(iaa.CropAndPad(
                percent=(-0.1, 0.1),
                pad_mode=ia.ALL,
                pad_cval=0
            )),
            sometimes(iaa.Affine(
                scale=(0.8, 1.2), # scale images to 90-110% of their size, individually per axis
                translate_percent={"x": (-0.1, 0.1), "y": (-0.1, 0.1)}, # translate by -20 to +20 percent (per axis)

                rotate=(-5, 5), # rotate by -5 to +5 degrees
                #order=[0, 1], # use nearest neighbour or bilinear interpolation (fast)
                cval=0, # if mode is constant, use a cval between 0 and 255
                #mode=ia.ALL, # use any of scikit-image's warping modes (see 2nd image from the top for examples)
                shear=(-1, 1) # shear by -16 to +16 degrees
            )),
            iaa.Resize({"height": h, "width": w}),
            iaa.AddToHueAndSaturation((-5, 5), name= "AddToHueAndSaturation"), # change the hue and saturation by up to 20
            iaa.Multiply((1, 1.5))
            ],
            random_order=False
        )
hooks_labels = ia.HooksImages(activator=activator_labels)

In [ ]:
import glob
imag_paths = sorted(glob.glob(os.path.join(base_data_dir, image_directory_name + "/*.jpg"), recursive=True) + glob.glob(os.path.join(base_data_dir, image_directory_name + "/*.bmp"), recursive=True))
mask_paths = sorted(glob.glob(os.path.join(base_data_dir, mask_directory_name + "/*.png"), recursive=True))
bkgd_paths = sorted(glob.glob(os.path.join(base_data_dir, background_dir_name + "/*.jpg"), recursive=True) + glob.glob(os.path.join(base_data_dir, background_dir_name + "/*.bmp"), recursive=True))

print(f'Total Train Images : {len(imag_paths)}')
print(f'Total Mask Image : {len(mask_paths)}')
print(f'Total background images : {len(bkgd_paths)}')

In [ ]:
train_imag_paths, test_imag_paths, train_mask_paths, test_mask_paths = train_test_split(imag_paths, mask_paths, test_size=0.1, random_state=42)

In [ ]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import cv2
from keras.utils import to_categorical

num_classes = 3
class ImageSource:
    def __init__(self, imag_paths, mask_paths, bkgd_paths, background_aug = False, backgroundchangeProbability = 0.25):
        self.imges = np.zeros((len(imag_paths), h, w, 3), dtype=np.float32)
        self.masks = np.zeros((len(mask_paths), h, w, num_classes), dtype=np.float32)
        self.backgroundChangeProbability = backgroundchangeProbability

        self.imag_paths = imag_paths
        self.mask_paths = mask_paths
        self.bkgd_paths = bkgd_paths
        
        self.total_images = len(imag_paths)

        self.max_blur_size = 10
        self.min_blur_size = 1
        self.max_circle_size = 0.11
        
        self.background_aug = background_aug
        self.find_class_weights()
        self.generate_augmented_images()

    def preprocess_input(self, input_im):
        return preprocess_input(input_im)
        
    def revert_preprocess(self, in_im):
        return revert_preprocess(in_im)

    def generate_augmented_images(self):
        class_weights = [0 for i in range(num_classes)]
        for n, (image_path, mask_path) in tqdm(enumerate(zip(self.imag_paths, self.mask_paths))):
            #print(os.path.split(image_path)[1], os.path.split(mask_path)[1])
            random_background_image_index = random.randint(0, len(bkgd_paths) - 1)

            imag_data = img_to_array(load_img(image_path)).astype(np.uint8)
            mask_data = img_to_array(load_img(mask_path, color_mode = "grayscale")).astype(np.uint8).squeeze()

            seq_deterministic = seq.to_deterministic() 
            imag_data_augmented = seq_deterministic.augment_image(imag_data)
            mask_data_augmented = seq_deterministic.augment_image(mask_data, hooks=hooks_labels)

            image_filename = os.path.split(image_path)[1]
            exclude_file = image_filename in file_names_to_exclude_from_bg_change
            has_mask = mask_data.max() > 0
            changeBackground = self.backgroundChangeProbability > random.random()
            if self.background_aug and has_mask and not exclude_file and changeBackground:
                bkgd_path = self.bkgd_paths[random_background_image_index]
                bkgd_data = img_to_array(load_img(bkgd_path)).astype(np.uint8)
                bkgd_data_augmented = seq_deterministic.augment_image(bkgd_data)
                expanded_mask = np.expand_dims(mask_data_augmented == 0, -1)
                mask = np.repeat(expanded_mask, 3, axis=-1)
                imag_data_augmented[mask] = bkgd_data_augmented[mask]
            image_blended_float = imag_data_augmented.astype(np.float32)
            
            self.imges[n] = self.preprocess_input(image_blended_float)
            mask_3d = to_categorical(mask_data_augmented, num_classes)
            self.masks[n] = mask_3d.astype(np.float32)

    def find_class_weights(self):
        class_weights = [0 for i in range(num_classes)]
        for n, mask_path in tqdm(enumerate(self.mask_paths)):
            mask_data = img_to_array(load_img(mask_path, color_mode = "grayscale")).astype(np.uint8).squeeze()
            total_pixels = mask_data.shape[0] * mask_data.shape[1]
            for i in range(num_classes):
                class_weight = np.count_nonzero(mask_data == i) / total_pixels
                class_weights[i] += class_weight
                
        self.class_weights = [1 - (class_weights[i] / self.total_images) for i in range(num_classes)]
        print(self.class_weights)

    def display_images(self, display_all = False):       
        indices = []
        if display_all:
            indices = range(0,len(self.imges)-1)
        else:
            random_index = random.randint(0, len(self.imges)-1)
            indices = range(random_index,random_index+1)
            
        for indice in indices:

            selected_image = self.imges[indice]
            selected_mask = np.argmax(self.masks[indice], axis=2)
            has_mask = selected_image.max() > 0

            fig, (input_figure, output_figure1, output_figure2, output_figure3) = plt.subplots(1, 4, figsize = (20, 15))
            reverted_preprocessed_image = self.revert_preprocess(selected_image)

            input_figure.imshow(reverted_preprocessed_image)
            if has_mask: # if salt
                # draw a boundary(contour) in the original image separating salt and non-salt areas
                input_figure.contour(selected_mask.squeeze(), colors = 'k', linewidths = 5, levels = [0.25])

            input_figure.set_title('Image')
            input_figure.set_axis_off()
            output_figure1.imshow(selected_mask.squeeze() > 0, cmap = 'gray')
            output_figure1.set_title('Mask Image1')
            output_figure1.set_axis_off()   
            output_figure2.imshow(selected_mask.squeeze() == 1, cmap = 'gray')
            output_figure2.set_title('Mask Image2')
            output_figure2.set_axis_off()   
            output_figure3.imshow(selected_mask.squeeze() == 2, cmap = 'gray')
            output_figure3.set_title('Mask Image3')
            output_figure3.set_axis_off()       
        

    def display_images_pred(self, predict_func):
        random_index = random.randint(0, len(self.imges)-1)
            
        selected_image = self.imges[random_index]
        selected_mask = np.argmax(self.masks[random_index], axis=-1)
        has_mask = selected_image.max() > 0

        fig, (input_figure, output_figure, pred_figure) = plt.subplots(1, 3, figsize = (20, 15))
        reverted_preprocessed_image = self.revert_preprocess(selected_image)

        input_figure.imshow(reverted_preprocessed_image)
        if has_mask:
            input_figure.contour(selected_mask.squeeze(), colors = 'k', linewidths = 5, levels = [0.25])

        input_figure.set_title('Image')
        input_figure.set_axis_off()

        output_figure.imshow(selected_mask.squeeze(), cmap = 'gray')
        if has_mask:
            output_figure.contour(selected_mask.squeeze(), colors = 'k', linewidths = 5, levels = [0.25])
        output_figure.set_title('Mask Image')
        output_figure.set_axis_off()   

        pred = np.argmax(predict_func(selected_image), axis=-1)
        pred_figure.imshow(pred.squeeze(), cmap = 'gray')
        pred_figure.set_title('Predicted Image')
        pred_figure.set_axis_off()          

# **Instantiate validation and training sources; visualize the data**

In [ ]:
train_source = ImageSource(train_imag_paths, train_mask_paths, bkgd_paths, background_aug = True)
test_source = ImageSource(test_imag_paths, test_mask_paths, bkgd_paths, background_aug = False)

In [ ]:
train_source.display_images(False)
train_source.display_images(False)
train_source.display_images(False)
train_source.display_images(False)
train_source.display_images(False)
train_source.display_images(False)
train_source.display_images(False)
train_source.display_images(False)
train_source.display_images(False)
train_source.display_images(False)
train_source.display_images(False)

In [ ]:

test_source.display_images(False)
test_source.display_images(False)
test_source.display_images(False)
test_source.display_images(False)
test_source.display_images(False)

# **Create upsample layer**

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Conv2DTranspose
import tensorflow.keras as keras
#-------------------------------------------------------------------------------
def DepthwisePointWise(inputs, filters, name, kernel_size=(3, 3), strides=(1, 1)):
    depthwise = DepthwiseConv2D(kernel_size=kernel_size, strides=strides, padding='same', depth_multiplier=1, name=f'{name}_depth')(inputs)
    pointwise = Conv2D(filters=filters, kernel_size=(1, 1), strides=(1, 1), padding='same', name=f'{name}_point')(depthwise)
    return pointwise

# **UNET model function**

In [ ]:
from tensorflow.keras.applications import MobileNetV2

def UnetMobilenetModel(image_width, image_height, num_classes = 2):
    inputs = Input((image_height, image_width, 3), name="mobilenet")
    
    encoder = MobileNetV2(input_tensor=inputs, weights="imagenet", include_top=False, alpha=0.35)
    
    for layer in encoder.layers:
        layer.trainable = False

    skip_connection_names = ["mobilenet", "block_1_expand_relu", "block_3_expand_relu", "block_6_expand_relu"]
    encoder_output = encoder.get_layer("block_13_expand_relu").output
    
    f = [16, 32, 48, 64]
    x = encoder_output
    for i in range(1, len(skip_connection_names)+1, 1):
        x_skip = encoder.get_layer(skip_connection_names[-i]).output
        x = Conv2DTranspose(filters=f[-i], kernel_size=(2,2), strides=(2,2), padding='same', use_bias = False)(x)
        x = BatchNormalization()(x)
        x = Concatenate()([x, x_skip])
        
        x = DepthwisePointWise(x, f[-i], name=f'depth1_{i}')
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        
        x = DepthwisePointWise(x, f[-i], name=f'depth2_{i}')
        x = BatchNormalization()(x)
        x = Activation("relu")(x)
        
    x = DepthwisePointWise(x, num_classes, name='final')
    x = BatchNormalization()(x)
    x = Activation("softmax")(x)
    
    model = Model(inputs, x)
    return model

In [ ]:
model = UnetMobilenetModel(w, h, num_classes)

In [ ]:
model.summary()

# **Loss function and training parameters; Train the model**

In [ ]:
from keras import backend as K
from tensorflow.keras.losses import categorical_crossentropy

BATCH_SIZE = 16
EPOCHES = 100000
RELOAD_ON_EVERY = 10000
EPOCH_PATIENCE = 1000
LEARNING_RATE_PATIENCE = 100
LERNING_RATE = 0.000001

K.clear_session()
class_weights = train_source.class_weights
def weighted_categorical_crossentropy(y_true, y_pred):
    # convert class weights dictionary to tensor
    class_weights_tensor = K.constant(class_weights)
    
    # calculate the cross-entropy loss for each sample
    per_sample_loss = categorical_crossentropy(y_true * class_weights_tensor, y_pred * class_weights_tensor)
    # calculate the weighted loss for each sample
    weighted_loss = K.sum(per_sample_loss)
    
    return weighted_loss

class_weights = train_source.class_weights

metrics = ["accuracy", tf.keras.metrics.AUC()]
model.compile(optimizer=Adam(learning_rate=LERNING_RATE), loss=weighted_categorical_crossentropy, metrics=metrics)
gc.collect()

In [ ]:
from keras.callbacks import Callback
               
class UpdateDataSample(Callback):
    def __init__(self, train_data_source, batch_size = 4, update_frequency=10):
        super(UpdateDataSample, self).__init__()
        self.batch_size = batch_size
        self.train_data_source = train_data_source
        self.update_frequency = update_frequency

    def on_epoch_end(self, epoch, logs={}):
        if (epoch + 1) % self.update_frequency == 0:
            self.train_data_source.generate_augmented_images()
            num_batches = self.train_data_source.total_images // self.batch_size
            for i in range(num_batches):
                batch_x = self.train_data_source.imges[i*self.batch_size:(i+1)*self.batch_size]
                batch_y = self.train_data_source.masks[i*self.batch_size:(i+1)*self.batch_size]
                self.model.train_on_batch(batch_x, batch_y)

In [ ]:
from keras.utils import Sequence

callbacks = [
    EarlyStopping(patience=EPOCH_PATIENCE, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=LEARNING_RATE_PATIENCE, min_lr=0.0001, verbose=1),
    ModelCheckpoint(f'{model_name}.h5', 
                    verbose=1, 
                    save_best_only=True,
                    save_weights_only=True),
    CSVLogger(f"{model_name}.csv"),
    TensorBoard(log_dir=f'./{model_name}_logs'),
    UpdateDataSample(train_source, BATCH_SIZE, RELOAD_ON_EVERY)
]

if os.path.exists(f'{model_name}.h5'):
    print("Existing model loaded", model_name)
    model.load_weights(f'{model_name}.h5')

results = model.fit(train_source.imges, train_source.masks, batch_size = BATCH_SIZE, epochs=EPOCHES, callbacks=callbacks, validation_data=(test_source.imges, test_source.masks), use_multiprocessing=True)

df_result = pd.DataFrame(results.history)
df_result.sort_values('val_loss', ascending=True, inplace = True)
df_result

plt.figure(figsize = (15,6))
plt.title("Learning curve")
plt.plot(results.history["loss"], label="loss")
plt.plot(results.history["val_loss"], label="val_loss")
plt.plot(np.argmin(results.history["val_loss"]), np.min(results.history["val_loss"]), marker="x", color="r", label="best model")
plt.xlabel("Epochs")
plt.ylabel("log_loss")
plt.legend();

plt.figure(figsize = (15,6))
plt.title("Learning curve")
plt.plot(results.history["accuracy"], label="Accuracy")
plt.plot(results.history["val_accuracy"], label="val_Accuracy")
plt.plot(np.argmax(results.history["val_accuracy"]), np.max(results.history["val_accuracy"]), marker="x", color="r", label="best model")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.legend();

# **Inference; Dump model to use in c++**

In [ ]:
model.load_weights(f'{model_name}.h5')

In [ ]:
model.evaluate(test_source.imges, test_source.masks, verbose=1)

# **Predictions on training set**

In [ ]:
def pred_func(image_in):
    image_expanded = np.expand_dims(image_in, axis = 0)
    return model.predict(image_expanded)
test_source.display_images_pred(pred_func)
test_source.display_images_pred(pred_func)
test_source.display_images_pred(pred_func)

train_source.display_images_pred(pred_func)
train_source.display_images_pred(pred_func)
train_source.display_images_pred(pred_func)

test_source.display_images_pred(pred_func)
test_source.display_images_pred(pred_func)
test_source.display_images_pred(pred_func)

train_source.display_images_pred(pred_func)
train_source.display_images_pred(pred_func)
train_source.display_images_pred(pred_func)

In [ ]:
import numpy as np
import os
from tensorflow.keras.layers import Activation, Concatenate, Add, ZeroPadding2D, BatchNormalization, Conv2D, Input, MaxPooling2D, Dropout, concatenate, UpSampling2D, ReLU, InputLayer, Conv2DTranspose, DepthwiseConv2D

epsilon = np.finfo(float).eps
lines_in_graph = []

model_data_dir = os.path.join(model_dir, model_name)
if not os.path.exists(model_data_dir):
    os.makedirs(model_data_dir)
    
for i, layer in enumerate(model.layers): 
    print(i, ":", layer.__class__.__name__, layer.name)
    items_to_write = [layer.__class__.__name__, layer.name]
    if isinstance(layer,InputLayer):
        inpu_shape = layer.input.shape
        items_to_write.append("")
        items_to_write.append(str(inpu_shape[2]))
        items_to_write.append(str(inpu_shape[1]))
        items_to_write.append(str(inpu_shape[3]))
    elif isinstance(layer.input, list):
        input_layer_names = []
        for i in range(len(layer.input)):
            inpu_layer_name = layer.input[i].name.split('/')[0]
            input_layer_names.append(inpu_layer_name)
        items_to_write.append("&".join(input_layer_names))
        if isinstance(layer,Add):
            for i in range(len(layer.input)):
                inpu_shape = layer.input[i].shape
                items_to_write.append(str(inpu_shape[2]))
                items_to_write.append(str(inpu_shape[1]))
                items_to_write.append(str(inpu_shape[3]))
        elif isinstance(layer,Concatenate): 
            for i in range(len(layer.input)):
                inpu_shape = layer.input[i].shape
                items_to_write.append(str(inpu_shape[2]))
                items_to_write.append(str(inpu_shape[1]))
                items_to_write.append(str(inpu_shape[3]))
        elif isinstance(layer,HeadWrapper):
            for i in range(len(layer.input)):
                inpu_shape = layer.input[i].shape
                items_to_write.append(str(inpu_shape[2]))
                items_to_write.append(str(inpu_shape[1]))
                items_to_write.append(str(inpu_shape[3]))
        else:
            print("---------------------------------")
            print("Error: Multi input layer not known")
            print("---------------------------------")            
    else:
        inpu_layer_name = layer.input.name.split('/')[0]
        items_to_write.append(inpu_layer_name)
        if isinstance(layer,Conv2D):
            weights = layer.weights[0]
            path_to_save = os.path.join(model_data_dir, layer.name + '_weights.npy')
            np.save(path_to_save, weights)             
            if layer.use_bias:
                bias = layer.weights[1]
                path_to_save = os.path.join(model_data_dir, layer.name + '_bias.npy')
                np.save(path_to_save, bias)
            layer_activation = layer.activation
            if layer_activation:
                items_to_write.append(layer_activation.__name__)
            else:
                print("Warning: Activation layer present : ", layer_activation.__name__)                
                print("---------------------------------")
            weights_shape = layer.weights[0].shape
            items_to_write.append(str(weights_shape[0]))
            items_to_write.append(str(weights_shape[1]))
            items_to_write.append(str(weights_shape[2]))
            items_to_write.append(str(weights_shape[3]))
            inpu_shape = layer.input.shape
            items_to_write.append(str(inpu_shape[2]))
            items_to_write.append(str(inpu_shape[1]))
            items_to_write.append(str(inpu_shape[3]))
            strides = layer.strides
            if strides[0] != strides[1]:
                print("---------------------------------")
                print("Error: Strides must be equal")
                print("---------------------------------")
            dilation_rates = layer.dilation_rate
            if dilation_rates[0] != dilation_rates[1] and dilation_rates[1] != 1:
                print("---------------------------------")
                print("Error: dilation_rates must be equal")
                print("---------------------------------")                
            items_to_write.append(str(strides[0]))
            items_to_write.append(str(layer.use_bias))
            items_to_write.append(str(layer.padding))            
        elif isinstance(layer,Conv2DTranspose):
            weights = layer.weights[0]
            path_to_save = os.path.join(model_data_dir, layer.name + '_weights.npy')
            np.save(path_to_save, weights)             
            if layer.use_bias:
                bias = layer.weights[1]
                path_to_save = os.path.join(model_data_dir, layer.name + '_bias.npy')
                np.save(path_to_save, bias)
            weights_shape = layer.weights[0].shape
            items_to_write.append(str(weights_shape[0]))
            items_to_write.append(str(weights_shape[1]))
            items_to_write.append(str(weights_shape[2]))
            items_to_write.append(str(weights_shape[3]))
            inpu_shape = layer.input.shape
            items_to_write.append(str(inpu_shape[2]))
            items_to_write.append(str(inpu_shape[1]))
            items_to_write.append(str(inpu_shape[3]))
            strides = layer.strides
            if strides[0] != strides[1]:
                print("---------------------------------")
                print("Error: Strides must be equal")
                print("---------------------------------")
            items_to_write.append(str(strides[0]))
            items_to_write.append(str(layer.use_bias))
        elif isinstance(layer,DepthwiseConv2D):   
            weights = layer.weights[0]
            path_to_save = os.path.join(model_data_dir, layer.name + '_weights.npy')
            np.save(path_to_save, weights)             
            if layer.use_bias:
                bias = layer.weights[1]
                path_to_save = os.path.join(model_data_dir, layer.name + '_bias.npy')
                np.save(path_to_save, bias)
            weights_shape = layer.weights[0].shape
            items_to_write.append(str(weights_shape[0]))
            items_to_write.append(str(weights_shape[1]))
            items_to_write.append(str(weights_shape[2]))
            inpu_shape = layer.input.shape
            items_to_write.append(str(inpu_shape[2]))
            items_to_write.append(str(inpu_shape[1]))
            items_to_write.append(str(inpu_shape[3]))
            strides = layer.strides
            if strides[0] != strides[1]:
                print("---------------------------------")
                print("Error: Strides must be equal")
                print("---------------------------------")
            items_to_write.append(str(strides[0]))
            items_to_write.append(str(layer.use_bias))
        elif isinstance(layer,BatchNormalization):
            weights = layer.get_weights()
            gamma = weights[0]
            beta = weights[1]
            moving_mean = weights[2]
            moving_variance = weights[3]
            a = gamma / np.sqrt(moving_variance + epsilon)
            b = - moving_mean * a + beta
            path_to_save = os.path.join(model_data_dir, layer.name + '_mean.npy')
            np.save(path_to_save, b)
            path_to_save = os.path.join(model_data_dir, layer.name + '_variance.npy')
            np.save(path_to_save, a) 

            inpu_shape = layer.input.shape
            items_to_write.append(str(inpu_shape[2]))
            items_to_write.append(str(inpu_shape[1]))
            items_to_write.append(str(inpu_shape[3]))
        elif isinstance(layer,MaxPooling2D):
            inpu_shape = layer.input.shape
            items_to_write.append(str(inpu_shape[2]))
            items_to_write.append(str(inpu_shape[1]))
            items_to_write.append(str(inpu_shape[3]))
        elif isinstance(layer,ZeroPadding2D):
            if(layer.padding[0][1] != layer.padding[1][1] and layer.padding[0][0] == 0 and layer.padding[1][0] != 0):
                print(layer.padding)
                print("---------------------------------")
                print("Error: This padding not supported") 
                print("---------------------------------")
            inpu_shape = layer.input.shape
            items_to_write.append(str(inpu_shape[2]))
            items_to_write.append(str(inpu_shape[1]))
            items_to_write.append(str(inpu_shape[3]))
            items_to_write.append(str(layer.padding[0][0]))
            items_to_write.append(str(layer.padding[0][1]))
            items_to_write.append(str(layer.padding[1][0]))
            items_to_write.append(str(layer.padding[1][1]))
        elif isinstance(layer, ReLU):
            maxValue = layer.max_value
            inpu_shape = layer.input.shape
            items_to_write.append(str(inpu_shape[2]))
            items_to_write.append(str(inpu_shape[1]))
            items_to_write.append(str(inpu_shape[3]))
            items_to_write.append(str(maxValue))
        elif isinstance(layer, Activation):
            activation_type = layer.get_config()['activation']
            maxValue = 0
            if activation_type == 'relu':
                items_to_write.append("relu")
                maxValue = 0
                if hasattr(layer,'max_value'):
                    maxValue = layer.max_value
                inpu_shape = layer.input.shape
                items_to_write.append(str(inpu_shape[2]))
                items_to_write.append(str(inpu_shape[1]))
                items_to_write.append(str(inpu_shape[3]))
                items_to_write.append(str(maxValue))
            elif activation_type == 'sigmoid':
                items_to_write.append("sigmoid")
                inpu_shape = layer.input.shape
                items_to_write.append(str(inpu_shape[2]))
                items_to_write.append(str(inpu_shape[1]))
                items_to_write.append(str(inpu_shape[3]))
            elif activation_type == 'softmax':
                items_to_write.append("softmax")
                inpu_shape = layer.input.shape
                output_shape = layer.output.shape
                print(output_shape)
                if len(output_shape) ==4: 
                    items_to_write.append(str(inpu_shape[2]))
                    items_to_write.append(str(inpu_shape[1]))
                    items_to_write.append(str(inpu_shape[3]))
                elif len(output_shape) == 3: 
                    items_to_write.append(str(inpu_shape[1]))
                    items_to_write.append(str(inpu_shape[2]))
                else:
                    print("---------------------------------")
                    print("Error: Unexpected shape ", activation_type)
                    print("---------------------------------") 
            else:
                print("---------------------------------")
                print("Error: Only Relu activation with max value supported, provided is ", activation_type)
                print("---------------------------------") 
        else:
            print("Error: Layer is not identified ", layer.__class__.__name__, layer.name)
    lines_in_graph.append(",".join(items_to_write))
with open(f'{model_name}.txt', 'w') as f:
    # Write each line of the list to the file
    for line in lines_in_graph:            
        f.write(line + '\n')
    print("File is saved to " + f'{model_name}.txt')    

# **Save the image in numpy format for testing in c++**

In [ ]:
image_file_path = os.path.join(sample_test_dir, "input.bmp")
selected_image = img_to_array(load_img(image_file_path).resize((w,h))).astype(np.uint8)

img_data = preprocess_input(selected_image)

numpy_in = os.path.join(sample_test_dir, "in_mobilenet.npy")
print(numpy_in)
np.save(numpy_in, np.transpose(img_data.copy().astype(np.float32),[1,0,2]))

def pred_funct(image_in, model_p):
    image_expanded = np.expand_dims(image_in, axis = 0)
    return model_p.predict(image_expanded)

out_img = pred_funct(img_data, model).squeeze()
fig, (input_figure, output_figure1, output_figure2, output_figure3) = plt.subplots(1, 4, figsize = (20, 15))

input_figure.imshow(selected_image)
input_figure.set_title('Image')
input_figure.set_axis_off()

selected_mask = np.argmax(out_img, axis=2)

input_figure.set_title('Image')
input_figure.set_axis_off()
output_figure1.imshow(selected_mask.squeeze(), cmap = 'gray')
output_figure1.set_title('Mask Image1')
output_figure1.set_axis_off()   
output_figure2.imshow(selected_mask.squeeze() == 1, cmap = 'gray')
output_figure2.set_title('Mask Image2')
output_figure2.set_axis_off()   
output_figure3.imshow(selected_mask.squeeze() == 2, cmap = 'gray')
output_figure3.set_title('Mask Image3')
output_figure3.set_axis_off()  

In [ ]:
def generate_model(model, check_index):
    input_layer_data = model.layers[0]
    intermediate_layer_data = model.layers[check_index-1]
    output_layer_data = model.layers[check_index]

    model1 = Model(input_layer_data.input, intermediate_layer_data.output)
    model2 = Model(intermediate_layer_data.output, output_layer_data.output)

    print(input_layer_data.name)
    print(intermediate_layer_data.name)
    print(output_layer_data.name)

    print(input_layer_data.input.shape[1:])
    print(intermediate_layer_data.output.shape[1:])
    print(output_layer_data.output.shape[1:])
    return model1, model2
def write_csv3d(output_data, file_name_sufix):  
    path_to_saved = os.path.join(sample_test_dir, file_name_sufix +"_python.csv")
    print(output_data.shape)
    # Loop over the rows and columns of the 3D array
    with open(path_to_saved, 'w') as file:
        for l in range(output_data.shape[3]):
            for k in range(output_data.shape[2]):
                for j in range(output_data.shape[1]):
                    for i in range(output_data.shape[0]):
                        # Join the text data in each row with commas
                        value = output_data[i, j, k, l]
                        # Write the row data to the output file
                        if i == output_data.shape[1] - 1:
                            file.write("{:.4f}".format(value))
                        else:
                            file.write("{:.4f},".format(value))
                    file.write("\n")
            
def write_csv(output_data, file_name_sufix):  
    path_to_saved = os.path.join(sample_test_dir, file_name_sufix +"_python.csv")
    print(output_data.shape)
    # Loop over the rows and columns of the 3D array
    with open(path_to_saved, 'w') as file:
        for k in range(output_data.shape[2]):
            for j in range(output_data.shape[0]):
                for i in range(output_data.shape[1]):
                    # Join the text data in each row with commas
                    value = output_data[j, i, k]
                    # Write the row data to the output file
                    if i == output_data.shape[1] - 1:
                        file.write("{:.4f}".format(value))
                    else:
                        file.write("{:.4f},".format(value))
                file.write("\n")
def runthis(mode1, model2, img_data, layer_name):
    intermediate_output = pred_funct(img_data, model1).squeeze()
    output = pred_funct(intermediate_output, model2).squeeze()
    print("-----------------")
    for k in range(3):
        for i in range(6):
            for j in range(6):
                print(img_data[i, j, k], end=", ")
            print()
        print("\n")
    print(img_data.shape)
    print("-----------------")
    for k in range(min(3, intermediate_output.shape[2])): 
        for i in range(6):
            for j in range(6):
                print(intermediate_output[i, j, k], end=", ")
            print()
        print("\n")
    print(intermediate_output.shape)
    write_csv(intermediate_output, layer_name +"_in")     
    print("-----------------")
    for k in range(min(3, output.shape[2])): 
        for i in range(6):
            for j in range(6):
                print(output[i, j, k], end=", ")
            print()
        print("\n")
    print(output.shape)  
    write_csv(output, layer_name + "out")                    

In [ ]:
print(len(model.layers))
check_index = 1
for index, layer in enumerate(model.layers):
    #print(index, layer.name)
    if layer.name in ["activation_26"]:
        print(index, layer.name)
        model1, model2 = generate_model(model, index)
        runthis(model1, model2, img_data, layer.name)         

In [ ]:
np.set_printoptions(precision=5, suppress=True)
filer_name = "expanded_conv_project_BN"
path_to_saved = os.path.join(model_data_dir, f'{filer_name}_mean.npy')
if os.path.exists(path_to_saved):
    data = np.load(path_to_saved)
    for i in range(min(5, data.size)):
        print(data[i], end=", ")
    print()
    print(data.shape)
    print("\n--------------")
    path_to_saved = os.path.join(model_data_dir, f'{filer_name}_variance.npy')
    data = np.load(path_to_saved)
    for i in range(min(5, data.size)):
        print(data[i], end=", ")
    print()
    print(data.shape)
    print("\n--------------")
else:
    path_to_saved = os.path.join(model_data_dir, f'{filer_name}_bias.npy')
    data = np.load(path_to_saved)
    for i in range(min(5, data.size)):
        print(data[i], end=", ")
    print()
    print(data.shape)
    print("\n--------------")
    
path_to_saved = os.path.join(model_data_dir, f'{filer_name}_weights.npy')
data = np.load(path_to_saved)
if 'depth' in filer_name:
    for i in range(3):
        for j in range(3):
            print(data[i, j, 0, 0], end=", ")
        print()
    print()
    for i in range(3):
        for j in range(3):
            print(data[i, j, 1, 0], end=", ")
        print()
else:
    for l in range(2):
        for k in range(4):
            for i in range(3):
                for j in range(3):
                    print(data[j, i, k ,l], end=", ")
                print()
            print("")
        print("+++++++++")
    print("\n--------------")
print(data.shape)
print("\n--------------")